In [19]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score

# Set global configurations
pd.set_option('display.max_columns', None)  # Display all columns in DataFrames

# Load datasets
def load_data():
    """Load and merge datasets."""
    customers = pd.read_csv("/content/Customers.csv")
    products = pd.read_csv("/content/Products.csv")
    transactions = pd.read_csv("/content/Transactions.csv")

    # Merge datasets
    merged_data = pd.merge(transactions, customers, on="CustomerID", how="left")
    merged_data = pd.merge(merged_data, products, on="ProductID", how="left")

    # Add a new column for transaction month
    merged_data['TransactionMonth'] = pd.to_datetime(merged_data['TransactionDate']).dt.to_period('M')

    return customers, products, transactions, merged_data

# Task 1: Exploratory Data Analysis (EDA)
def perform_eda(merged_data, customers):
    """Perform exploratory data analysis and generate visualizations."""
    # 1. Revenue by Region (Enhanced Bar Chart)
    revenue_by_region = merged_data.groupby('Region')['TotalValue'].sum().sort_values(ascending=False).reset_index()
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=revenue_by_region['Region'],
        y=revenue_by_region['TotalValue'],
        marker_color=px.colors.qualitative.Plotly,  # Custom color palette
        text=revenue_by_region['TotalValue'],  # Add text labels
        textposition='outside',  # Position text above the bars
        hovertext=[f"Revenue: ${val:,.2f}" for val in revenue_by_region['TotalValue']]  # Hover text
    ))
    fig.update_layout(
        title="Revenue by Region (Interactive Bar Chart)",
        xaxis_title="Region",
        yaxis_title="Total Revenue",
        template="plotly_white"
    )
    fig.show()

    # 2. Top 10 Products by Sales (Enhanced Bar Chart)
    top_products = merged_data.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).head(10).reset_index()
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=top_products['Quantity'],
        y=top_products['ProductName'],
        orientation='h',  # Horizontal bar chart
        marker_color=px.colors.sequential.Viridis,  # Custom color palette
        text=top_products['Quantity'],  # Add text labels
        textposition='outside',  # Position text above the bars
        hovertext=[f"Quantity Sold: {val}" for val in top_products['Quantity']]  # Hover text
    ))
    fig.update_layout(
        title="Top 10 Products by Sales (Interactive Bar Chart)",
        xaxis_title="Quantity Sold",
        yaxis_title="Product Name",
        template="plotly_white"
    )
    fig.show()

    # 3. Monthly Revenue Trend (Line Chart)
    monthly_revenue = merged_data.groupby('TransactionMonth')['TotalValue'].sum().reset_index()
    monthly_revenue['TransactionMonth'] = monthly_revenue['TransactionMonth'].astype(str)  # Convert to string
    fig = px.line(monthly_revenue, x='TransactionMonth', y='TotalValue', title="Monthly Revenue Trend", markers=True)
    fig.update_traces(line_color='blue', line_width=2.5)  # Customize line color and width
    fig.show()

    # 4. Customer Acquisition Over Time (Enhanced Bar Chart)
    customers['SignupYear'] = pd.to_datetime(customers['SignupDate']).dt.year
    signup_trend = customers.groupby('SignupYear').size().reset_index(name='Signups')
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=signup_trend['SignupYear'],
        y=signup_trend['Signups'],
        marker_color=px.colors.sequential.Plasma,  # Custom color palette
        text=signup_trend['Signups'],  # Add text labels
        textposition='outside',  # Position text above the bars
        hovertext=[f"Signups: {val}" for val in signup_trend['Signups']]  # Hover text
    ))
    fig.update_layout(
        title="Customer Acquisition Over Time (Interactive Bar Chart)",
        xaxis_title="Year",
        yaxis_title="Number of Signups",
        template="plotly_white"
    )
    fig.show()

    # 5. Average Transaction Value by Region (Enhanced Bar Chart)
    avg_transaction_value = merged_data.groupby('Region')['TotalValue'].mean().sort_values(ascending=False).reset_index()
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=avg_transaction_value['Region'],
        y=avg_transaction_value['TotalValue'],
        marker_color=px.colors.sequential.Turbo,  # Custom color palette
        text=[f"${val:,.2f}" for val in avg_transaction_value['TotalValue']],  # Add text labels
        textposition='outside',  # Position text above the bars
        hovertext=[f"Avg Value: ${val:,.2f}" for val in avg_transaction_value['TotalValue']]  # Hover text
    ))
    fig.update_layout(
        title="Average Transaction Value by Region (Interactive Bar Chart)",
        xaxis_title="Region",
        yaxis_title="Average Transaction Value",
        template="plotly_white"
    )
    fig.show()

# Task 2: Lookalike Model
def build_lookalike_model(merged_data, customers):
    """Build a lookalike model to recommend similar customers."""
    # Feature Engineering for Customer Profiles
    customer_profiles = merged_data.groupby('CustomerID').agg(
        TotalSpend=('TotalValue', 'sum'),
        AvgTransactionValue=('TotalValue', 'mean'),
        FavoriteCategory=('Category', lambda x: x.mode()[0]),
        TransactionCount=('TransactionID', 'count')
    ).reset_index()

    # Merge with customer demographics
    customer_profiles = pd.merge(customer_profiles, customers, on="CustomerID", how="left")

    # One-Hot Encoding for categorical variables
    encoder = OneHotEncoder()
    encoded_region = encoder.fit_transform(customer_profiles[['Region']])
    encoded_region_df = pd.DataFrame(encoded_region.toarray(), columns=encoder.get_feature_names_out(['Region']))

    # Combine features
    features = pd.concat([customer_profiles[['TotalSpend', 'AvgTransactionValue', 'TransactionCount']], encoded_region_df], axis=1)

    # Normalize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Calculate similarity scores using cosine similarity
    similarity_matrix = cosine_similarity(scaled_features)

    # Generate lookalike recommendations
    lookalike_results = {}
    for i in range(20):  # First 20 customers
        customer_id = customer_profiles.iloc[i]['CustomerID']
        similarity_scores = similarity_matrix[i]
        top_3_indices = similarity_scores.argsort()[-4:-1][::-1]  # Exclude self
        top_3_customers = customer_profiles.iloc[top_3_indices]['CustomerID'].values
        top_3_scores = similarity_scores[top_3_indices]
        lookalike_results[customer_id] = list(zip(top_3_customers, top_3_scores))

    # Save results to CSV
    lookalike_df = pd.DataFrame(lookalike_results.items(), columns=['CustomerID', 'Lookalikes'])
    lookalike_df.to_csv("Lookalike.csv", index=False)
    print("Lookalike model results saved to 'Lookalike.csv'.")

# Task 3: Customer Segmentation
def perform_customer_segmentation(scaled_features):
    """Perform customer segmentation using clustering."""
    # Use K-Means clustering
    kmeans = KMeans(n_clusters=4, random_state=42)
    clusters = kmeans.fit_predict(scaled_features)

    # Evaluate clustering using Davies-Bouldin Index
    db_index = davies_bouldin_score(scaled_features, clusters)
    print(f"Davies-Bouldin Index: {db_index}")

    # Visualize clusters using PCA
    pca = PCA(n_components=2)
    pca_features = pca.fit_transform(scaled_features)
    cluster_df = pd.DataFrame(pca_features, columns=['PCA1', 'PCA2'])
    cluster_df['Cluster'] = clusters

    fig = px.scatter(cluster_df, x='PCA1', y='PCA2', color='Cluster', title="Customer Segmentation Clusters", labels={'PCA1': 'PCA Component 1', 'PCA2': 'PCA Component 2'})
    fig.show()

# Main function to execute all tasks
def main():
    # Load data
    customers, products, transactions, merged_data = load_data()

    # Task 1: Perform EDA
    print("Performing Exploratory Data Analysis (EDA)...")
    perform_eda(merged_data, customers)

    # Task 2: Build Lookalike Model
    print("Building Lookalike Model...")
    build_lookalike_model(merged_data, customers)

    # Task 3: Perform Customer Segmentation
    print("Performing Customer Segmentation...")
    customer_profiles = merged_data.groupby('CustomerID').agg(
        TotalSpend=('TotalValue', 'sum'),
        AvgTransactionValue=('TotalValue', 'mean'),
        TransactionCount=('TransactionID', 'count')
    ).reset_index()
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(customer_profiles[['TotalSpend', 'AvgTransactionValue', 'TransactionCount']])
    perform_customer_segmentation(scaled_features)

# Execute the main function
if __name__ == "__main__":
    main()


Performing Exploratory Data Analysis (EDA)...


Building Lookalike Model...
Lookalike model results saved to 'Lookalike.csv'.
Performing Customer Segmentation...
Davies-Bouldin Index: 1.060424039992303
